# Scaling of VCT/captive

In [ ]:
# %load imports.py
# %load imports.py
## Local packages:

%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

## External packages:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
np.set_printoptions(linewidth=150)

import numpy as np
import os
import matplotlib.pyplot as plt
#if os.name == 'nt':
#    plt.style.use('presentation.mplstyle')  # Windows

import plotly.express as px 
import plotly.graph_objects as go

import seaborn as sns
import sympy as sp
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                      Particle, Point)
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify

import pyro

import sklearn
import pykalman
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import statsmodels.api as sm

from scipy.integrate import solve_ivp

## Local packages:
from vessel_manoeuvring_models.data import mdl

from vessel_manoeuvring_models.symbols import *
from vessel_manoeuvring_models.parameters import *
import vessel_manoeuvring_models.symbols as symbols
from vessel_manoeuvring_models import prime_system
from vessel_manoeuvring_models.models import regression
from vessel_manoeuvring_models.visualization.regression import show_pred
from vessel_manoeuvring_models.visualization.plot import track_plot

## Load models:
# (Uncomment these for faster loading):
import vessel_manoeuvring_models.models.vmm_martin  as vmm 
from vessel_manoeuvring_models.models.vmm import ModelSimulator
from example_ship2 import ship_parameters, df_parameters

from vessel_manoeuvring_models.vct_scaling import scale_force_to_model_scale, scale_moment_to_model_scale

if os.name == 'nt':
    plt.style.use('../docs/book/book.mplstyle')  # Windows
    
from vessel_manoeuvring_models.visualization.plot import track_plot, plot, captive_plot

In [ ]:
df_VCT = pd.read_csv('../data/external/vct.csv', index_col=0)
df_VCT.head()

In [ ]:
df_captive = pd.read_csv('../data/external/captive.csv', sep=';', index_col=0)
df_captive.head()

In [ ]:
data_scaled = df_VCT.copy()
ship_data = ship_parameters
ship_data['scale_factor'] = scale_factor = 41.2
keys_forces = ["fx", "fy", "fz"]
scale_factor = ship_data["scale_factor"]
forces = data_scaled[keys_forces]
data_scaled[keys_forces] = scale_force_to_model_scale(
    forces=forces, scale_factor=scale_factor, **data_scaled
)


keys_moments = ["mx", "my", "mz"]
data_scaled[keys_moments] = scale_moment_to_model_scale(
    data_scaled[keys_moments], scale_factor=scale_factor, **data_scaled
)

data_scaled[['u','v','V']]/=np.sqrt(scale_factor)

In [ ]:
df_captive_scaled = df_captive.copy()

keys_forces = ["fx", "fy"]

alpha = float(df_captive_scaled.iloc[0]['ScaleFactor'])

df_captive_scaled[keys_forces] = df_captive_scaled[keys_forces]*(alpha**3)/(scale_factor**3)

keys_moments = ["mx", "mz"]
df_captive_scaled[keys_moments] = df_captive_scaled[keys_moments]*(alpha**4)/(scale_factor**4)


df_captive_scaled[['u','v','V']]*=np.sqrt(alpha)/np.sqrt(scale_factor)


df_captive_scaled['item'] = 'captive'


In [ ]:
df_captive_scaled['V'].max()

In [ ]:
data_scaled['V'].max()

In [ ]:
data_scaled['model_name'].unique()

In [ ]:
models = data_scaled.groupby('model_name')
V2_5_MDL_modelScale = models.get_group('V2_5_MDL_modelScale')
V2_5_R1 = models.get_group('V2_5_R2')



In [ ]:
data = V2_5_MDL_modelScale.copy()
data = data.append(V2_5_R1)
data['item'] = data['model_name']
data = data.append(df_captive_scaled)

In [ ]:
import plotly.express as px
from vessel_manoeuvring_models.visualization.plot import test_type_xplot

data['V_round'] = data['V'].round(1)
V = data.groupby(by='V_round')['item'].count().sort_values().index[-1]
data_V = data.groupby(by='V_round').get_group(V).copy()

data_V["v*r"] = data_V["v"] * data_V["r"]
data_V["beta"] = -np.arctan2(data_V["v"], data_V["u"])

key = 'mz'
for test_type, df in data_V.groupby(by='test type'):
    
    x = test_type_xplot.get(test_type, 'delta')
    
    fig = px.line(df.sort_values(by=x), x=x, y=key, color="item", width=1200, height=300, title=test_type)
    
    fig.update_layout(yaxis_range=[data_V[key].min(),data_V[key].max()])
    
    fig.show()

In [ ]:
data['test type'].unique()

In [ ]:
df_resistance = data.groupby(by='test type').get_group('resistance')
x = 'V'
key = 'fx'
fig = px.line(df_resistance.sort_values(by=x), x=x, y=key, color="item", width=1200, height=300, title=test_type)
   
    
fig.show()